<a href="https://colab.research.google.com/github/Freaker99/car-detection/blob/main/car_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import os

drive.flush_and_unmount()
drive.mount('/content/drive')
root = '/content/drive/My Drive/car-detection/'

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
!pip install opencv-python
!pip install torch torchvision
!pip install numpy

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt

In [ ]:
# Importowanie niezbędnych bibliotek
import os
import torch
from torchvision.datasets import VisionDataset
from PIL import Image

# Ścieżki do plików etykiet
train_labels_file = "/labels/bdd100k_labels_images_train.json"
val_labels_file = "/labels/bdd100k_labels_images_val.json"

# Ścieżka do katalogu, w którym znajdują się obrazy
image_dir = "/images/"

In [ ]:
# Klasa datasetu BDD100K
class BDD100KDataset(VisionDataset):
    def __init__(self, labels_file, image_dir, transform=None, target_transform=None):
        super(BDD100KDataset, self).__init__(image_dir, transform=transform, target_transform=target_transform)
        self.labels = self._read_labels(labels_file)

    def __getitem__(self, index):
        image_path = os.path.join(self.root, self.images[index])
        image = Image.open(image_path).convert("RGB")
        label = self.labels[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.labels)

    def _read_labels(self, labels_file):
        with open(labels_file, "r") as f:
            labels = f.read().splitlines()
        return labels

# Tworzenie instancji datasetu treningowego i walidacyjnego
train_dataset = BDD100KDataset(train_labels_file, image_dir)
val_dataset = BDD100KDataset(val_labels_file, image_dir)

# Definiowanie DataLoader dla treningu i walidacji
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

# Trenowanie modelu YOLOv5
# Tutaj dodaj kod trenujący model YOLOv5 na danych BDD100K, np. importowanie YOLOv5, konfiguracja treningu, pętla trenująca itp.

# Przykładowa pętla trenująca
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YourYOLOv5Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = YourLossFunction()

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Ewaluacja na danych walidacyjnych
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        accuracy = total_correct / total_samples
        print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy}")

# Tutaj dodaj kod do zapisania wytrenowanego modelu YOLOv5